In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy
import random

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA

from keras.models import Sequential, Model , load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, PReLU, GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical, Sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169
#import tensorflow as tf
from skimage.transform import AffineTransform, warp

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics
import tensorflow_hub as hub

from efficientnet.keras import EfficientNetB6

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [4]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [5]:
y = y.set_index("image_id")

In [6]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [7]:
def affine_image(img):
    """

    Args:
        img: (h, w) or (1, h, w)

    Returns:
        img: (h, w)
    """
    # ch, h, w = img.shape
    # img = img / 255.
    if img.ndim == 3:
        img = img[0]

    # --- scale ---
    min_scale = 0.8
    max_scale = 1.2
    sx = np.random.uniform(min_scale, max_scale)
    sy = np.random.uniform(min_scale, max_scale)

    # --- rotation ---
    max_rot_angle = 7
    rot_angle = np.random.uniform(-max_rot_angle, max_rot_angle) * np.pi / 180.

    # --- shear ---
    max_shear_angle = 10
    shear_angle = np.random.uniform(-max_shear_angle, max_shear_angle) * np.pi / 180.

    # --- translation ---
    max_translation = 4
    tx = np.random.randint(-max_translation, max_translation)
    ty = np.random.randint(-max_translation, max_translation)

    tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))
    transformed_image = warp(img, tform)
    assert transformed_image.ndim == 2
    return (transformed_image*255).astype(np.uint8)

In [8]:
size=(120, 120)

def calcRotate(img):
    detector = cv2.ORB_create()
    keypoints=detector.detect(img)
    descriptors=detector.compute(img,keypoints)
    angles=[]
    weights=[]
    for i in descriptors[0]:
        if i.angle!=-1:
            angles.append(i.angle)
            weights.append(i.response)
    if len(angles)==0:
        return 0
    else:
        return np.average(angles,weights=weights)

def upper(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[up:mid,:])==0:
            up=mid
        else:
            bottom=mid
    return max(up-mergin,0)

def lower(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[mid:bottom,:])==0:
            bottom=mid
        else:
            up=mid
    return min(bottom+mergin,img.shape[0])

def lefter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,left:mid])==0:
            left=mid
        else:
            right=mid
    return max(left-mergin,0)

def righter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,mid:right])==0:
            right=mid
        else:
            left=mid
    return min(right+mergin,img.shape[1])

def transformImg(img,size=(255,255),training=True,clahe=False):
    ret2, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    img = 255-img
    if training:
        img=affine_image(img)
    if clahe:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img = clahe.apply(img)
    img = img[upper(img):lower(img),lefter(img):righter(img)]
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img, (size[0],size[1]))
    kernel = np.ones((3,3),np.float32)/9
    img = cv2.filter2D(img,-1,kernel)
    return img

def randomErase(img, prob=True):
    # random erasing
    # https://github.com/yu4u/cutout-random-erasing
    p = 0.5
    s_l = 0.02
    s_h = 0.4
    r_1 = 0.3
    r_2 = 1 / 0.3
    v_l = 0
    v_h = 255
    input_size=size[0]
    if prob==False or np.random.random()<p:

        while True:
            s = np.random.uniform(s_l, s_h) * input_size * input_size
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, input_size)
            top = np.random.randint(0, input_size)
            if left + w <= input_size and top + h <= input_size:
                break
        c = np.random.uniform(v_l, v_h, (h, w, 3))
        img[top : top + h, left : left + w, :] = c
    return img

In [9]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64,size=(255,255),alpha=3,clahe=True):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y
        self.size=size
        self.alpha=alpha
        self.clahe=clahe

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, idx):
        
        _imgs=self.X[idx * self.batch_size:(idx + 1) * self.batch_size,:,:]
        
            
        
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img,size=self.size,training=self.training,clahe=self.clahe))
         
        
        
        imgs=np.asarray(imgs)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[idx * self.batch_size:(idx + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
    
        #mix up
        if self.training :
            r = np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            ratio=np.random.beta(self.alpha,self.alpha,imgs.shape[0])
            ratio[ratio>1]=1
            ratio[ratio<0]=0
            imgs=np.tile(ratio,(3,*size,1)).T*imgs+np.tile((1-ratio),(3,*size,1)).T*imgs2
            grapheme=np.tile(ratio,(168,1)).T*grapheme+np.tile((1-ratio),(168,1)).T*grapheme2
            vowel=np.tile(ratio,(11,1)).T*vowel+np.tile((1-ratio),(11,1)).T*vowel2
            consonant=np.tile(ratio,(7,1)).T*consonant+np.tile((1-ratio),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
   
        if self.training:
            imgs = [randomErase(img) for img in imgs]
            pass
            
            
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y



In [10]:
def getMultiBased():
    model =  EfficientNetB6(weights="imagenet", include_top=False,input_shape=(*size,3))
    x = model.output  
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    #x = Dense(1024, activation="relu")(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236),y,train_size=0.9,random_state=8000)
del df

In [12]:
early_stopping =  EarlyStopping(monitor='val_loss', min_delta=0.0, patience=20)
checkpoint = ModelCheckpoint(filepath="tmp-effb6-epoch{epoch:04}.h5")

In [ ]:
batch_size = 64
train_gen = DataLoader(X_train, y_train, training=True, batch_size=batch_size,size=size,clahe=False,alpha=3)
valid_gen = DataLoader(X_test, y_test, training=False, batch_size=batch_size,size=size,clahe=False,alpha=3)
#strategy = tf.distribute.MirroredStrategy()
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
#with strategy.scope():
#model = getMultiBased()
model=load_model("tmp-effb6-epoch0003.h5")
model.summary()
#model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[0.5,0.25,0.25])
print("compiled")
model.fit_generator(train_gen, validation_data=valid_gen, epochs=80, callbacks=[checkpoint], workers=8, use_multiprocessing=True)
model.save("multiEfficientB6.h5")

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120, 120, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 60, 60, 56)   1512        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 60, 60, 56)   224         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 60, 60, 56)   0           stem_bn[0][0]                    
____________________________________________________________________________________________

Epoch 1/80
2825/2825 [==============================] - 1874s 663ms/step - loss: 1.0260 - dense_1_loss: 1.4093 - dense_2_loss: 0.7673 - dense_3_loss: 0.5182 - dense_1_acc: 0.6548 - dense_2_acc: 0.8063 - dense_3_acc: 0.8619 - val_loss: 0.1389 - val_dense_1_loss: 0.1752 - val_dense_2_loss: 0.0606 - val_dense_3_loss: 0.0575 - val_dense_1_acc: 0.9538 - val_dense_2_acc: 0.9853 - val_dense_3_acc: 0.9833
Epoch 2/80
2825/2825 [==============================] - 1753s 621ms/step - loss: 1.0143 - dense_1_loss: 1.3903 - dense_2_loss: 0.7610 - dense_3_loss: 0.5158 - dense_1_acc: 0.6581 - dense_2_acc: 0.8094 - dense_3_acc: 0.8643 - val_loss: 0.0969 - val_dense_1_loss: 0.1636 - val_dense_2_loss: 0.0527 - val_dense_3_loss: 0.0542 - val_dense_1_acc: 0.9554 - val_dense_2_acc: 0.9872 - val_dense_3_acc: 0.9860
Epoch 3/80
2825/2825 [==============================] - 1752s 620ms/step - loss: 1.0065 - dense_1_loss: 1.3782 - dense_2_loss: 0.7569 - dense_3_loss: 0.5128 - dense_1_acc: 0.6589 - dense_2_acc: 0.81

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 1752s 620ms/step - loss: 0.9519 - dense_1_loss: 1.2920 - dense_2_loss: 0.7300 - dense_3_loss: 0.4937 - dense_1_acc: 0.6760 - dense_2_acc: 0.8243 - dense_3_acc: 0.8765 - val_loss: 0.1009 - val_dense_1_loss: 0.1641 - val_dense_2_loss: 0.0538 - val_dense_3_loss: 0.0478 - val_dense_1_acc: 0.9556 - val_dense_2_acc: 0.9879 - val_dense_3_acc: 0.9877
Epoch 12/80
2279/2825 [=======================>......] - ETA: 5:30 - loss: 0.9461 - dense_1_loss: 1.2828 - dense_2_loss: 0.7263 - dense_3_loss: 0.4922 - dense_1_acc: 0.6783 - dense_2_acc: 0.8254 - dense_3_acc: 0.8753

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

2825/2825 [==============================] - 1758s 622ms/step - loss: 0.9031 - dense_1_loss: 1.2138 - dense_2_loss: 0.7065 - dense_3_loss: 0.4781 - dense_1_acc: 0.6932 - dense_2_acc: 0.8358 - dense_3_acc: 0.8841 - val_loss: 0.1639 - val_dense_1_loss: 0.1440 - val_dense_2_loss: 0.0489 - val_dense_3_loss: 0.0469 - val_dense_1_acc: 0.9602 - val_dense_2_acc: 0.9881 - val_dense_3_acc: 0.9881
Epoch 24/80
1839/2825 [==================>...........] - ETA: 9:57 - loss: 0.9019 - dense_1_loss: 1.2115 - dense_2_loss: 0.7055 - dense_3_loss: 0.4788 - dense_1_acc: 0.6926 - dense_2_acc: 0.8364 - dense_3_acc: 0.8840

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 1754s 621ms/step - loss: 0.8903 - dense_1_loss: 1.1923 - dense_2_loss: 0.7020 - dense_3_loss: 0.4744 - dense_1_acc: 0.6982 - dense_2_acc: 0.8392 - dense_3_acc: 0.8853 - val_loss: 0.1211 - val_dense_1_loss: 0.1444 - val_dense_2_loss: 0.0474 - val_dense_3_loss: 0.0463 - val_dense_1_acc: 0.9615 - val_dense_2_acc: 0.9875 - val_dense_3_acc: 0.9890
Epoch 28/80
1809/2825 [==================>...........] - ETA: 10:13 - loss: 0.8867 - dense_1_loss: 1.1872 - dense_2_loss: 0.6995 - dense_3_loss: 0.4729 - dense_1_acc: 0.6989 - dense_2_acc: 0.8393 - dense_3_acc: 0.8856

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 1758s 622ms/step - loss: 0.8806 - dense_1_loss: 1.1771 - dense_2_loss: 0.6955 - dense_3_loss: 0.4726 - dense_1_acc: 0.7030 - dense_2_acc: 0.8422 - dense_3_acc: 0.8865 - val_loss: 0.1058 - val_dense_1_loss: 0.1541 - val_dense_2_loss: 0.0562 - val_dense_3_loss: 0.0521 - val_dense_1_acc: 0.9610 - val_dense_2_acc: 0.9874 - val_dense_3_acc: 0.9876
Epoch 32/80
2825/2825 [==============================] - 1758s 622ms/step - loss: 0.8770 - dense_1_loss: 1.1719 - dense_2_loss: 0.6937 - dense_3_loss: 0.4705 - dense_1_acc: 0.7017 - dense_2_acc: 0.8420 - dense_3_acc: 0.8881 - val_loss: 0.0583 - val_dense_1_loss: 0.1477 - val_dense_2_loss: 0.0512 - val_dense_3_loss: 0.0456 - val_dense_1_acc: 0.9617 - val_dense_2_acc: 0.9879 - val_dense_3_acc: 0.9882
Epoch 33/80
2825/2825 [==============================] - 1834s 649ms/step - loss: 0.8749 - dense_1_loss: 1.1682 - dense_2_loss: 0.6945 - dense_3_loss: 0.4688 - dense_1_acc: 0.7030 - dense_2_acc: 0.8431 - dens

In [ ]:
model.evaluate(valid_gen)